## Setup

Load required packages

In [55]:
import pandas as pd
import io
import requests
import numpy as np
import random
import libpysal
import pysal

Read data

In [81]:
#Read from URL (Doesnt work for private repos)
#url="https://github.com/konstantinklemmer/spacegan/raw/master/data/synth_data.csv"
#s=requests.get(url).content
#data=pd.read_csv(io.StringIO(s.decode('utf-8')))

#Read from local file
data=pd.read_csv("C:/Users/Konstantin Klemmer/Documents/GitHub/spacegan/data/grid_aug.csv")

In [82]:
data.head(5)

,id,y,longitude,latitude,z,nn0,nn1,nn2,nn3,nn4,...,nn20,nn21,nn22,nn23,nn24,nn25,nn26,nn27,nn28,nn29
0,0,-0.844504,2.5,2.5,1.370958,-0.832568,-0.832449,-0.845500,-0.793995,-0.794071,...,-0.794673,-0.527385,-0.526857,-0.832357,-0.832634,-0.641253,-0.641905,-0.730428,-0.731069,-0.845475
1,1,-0.832568,7.5,2.5,-0.564698,-0.844504,-0.845500,-0.793995,-0.832449,-0.832383,...,-0.641905,-0.794130,-0.641186,-0.832634,-0.832357,-0.730428,-0.641253,-0.793685,-0.845475,-0.387048
2,2,-0.793995,12.5,2.5,0.363128,-0.832568,-0.730340,-0.832383,-0.845500,-0.793699,...,-0.832634,-0.730428,-0.387048,-0.794130,-0.730511,-0.527461,-0.832357,-0.793685,-0.641186,-0.845475
3,3,-0.730340,17.5,2.5,0.632863,-0.793699,-0.793995,-0.641493,-0.730159,-0.832383,...,-0.832634,-0.794071,-0.793685,-0.794082,-0.642062,-0.222415,-0.832357,-0.388055,-0.845475,-0.794130
4,4,-0.641493,22.5,2.5,0.404268,-0.730159,-0.730340,-0.527385,-0.641905,-0.793699,...,-0.844502,-0.526827,-0.832475,-0.832016,-0.730473,-0.844504,-0.031306,-0.845475,-0.832357,-0.832004


Create a pairwise distance matrix (Euclidean) between the points

In [83]:
dist = pysal.lib.cg.distance_matrix(np.array(data[["longitude","latitude"]]))

Get the `k` smallest distances (of the whole matrix, since the points are equally distributed)

In [84]:
k=10
u_dist = np.unique(dist)
k_min_dist = np.sort(u_dist.flatten())[:k]

In [85]:
k_min_dist

array([ 0.        ,  5.        ,  7.07106781, 10.        , 11.18033989,
       14.14213562, 15.        , 15.8113883 , 18.02775638, 20.        ])

Create spatial points object ([KDTree](https://pysal.readthedocs.io/en/dev/library/cg/kdtree.html))). 

In [86]:
import pysal.lib
kd = pysal.lib.cg.kdtree.KDTree(np.array(data[["longitude","latitude"]]))

Compute spatial neighbourhoods weight matrix by distance threshold ("radius")


In [87]:
#wnn = pysal.lib.weights.KNN(kd, 8, ids=data["id"]) #KNN based weights
#wdist= pysal.lib.weights.distance.DistanceBand(kd, threshold=k_min_dist[1],binary=False,p=2) #Rook
wdist= pysal.lib.weights.distance.DistanceBand(kd, threshold=k_min_dist[2],binary=True,p=2) #Queen
#wdist= pysal.lib.weights.distance.DistanceBand(kd, threshold=k_min_dist[4],binary=True,p=2) #Queen 2nd degree

## Spatial CV: Lat/Lon slicing

Create labels based on latitude / longitude binning and add the labels to the original data

In [88]:
data["lon_group"] = pd.cut(data["longitude"],bins=5,labels=[1,2,3,4,5])
data["lat_group"] = pd.cut(data["latitude"],bins=5,labels=[1,2,3,4,5])

This method loops over our lat/lon groups, keeping each group as test data and the rest as train data. However, as we want to do spatial cross-validation, we remove neighbors of the test set. This can help to prevent model overfitting. Here, we remove 1st and 2nd degree neighbors, but the method can be adapted as needed. We create 10 folds (5 lon, 5 lat slicing) and save these in the columns `lat_group[1-5]` and `lon_group[1-5]`. For the values in each of these columns, `1` indicates testing data, `2` training data and `0`indicates data to be removed.

In [90]:
for q in list(data)[-2::]: #Loop over the two slicing label columns 
    data["s_id"] = data[q] #Define which label column to use for slicing
    
    for j in np.unique(data["s_id"]): #Loop over the unique labels in the slicing column 
        
        data[q+str(j)] = 0
        
        test = data[data["s_id"]==j] #Define test data 
        data.loc[data["id"].isin(np.array(test["id"])),q+str(j)] = 1
        
        temp_id = [] #Create empty neighbourhood index
        
        for k in test.index: #Fill neighborhood index using first degree neighbors of test data
            temp_id = np.unique(np.concatenate([temp_id,wdist.neighbors[k]]).ravel().astype(np.int32))
            
        for l in temp_id: #Include second degree neighbors
            temp_id = np.unique(np.concatenate([temp_id,wdist.neighbors[l]]).ravel().astype(np.int32))
        
        #for m in temp_id: #Include third degree neighbors
        #    temp_id = np.unique(np.concatenate([temp_id,wdist.neighbors[m]]).ravel().astype(np.int32))
            
        train = data[data["s_id"]!=j] #Define train data 
        train = train.drop(temp_id,errors="ignore") #Exclude neighbors from index
        data.loc[data["id"].isin(np.array(train["id"])),q+str(j)] = 2
        
        #INSERT DATA AUGMENTATION METHOD HERE
        #train_aug = ...
        
        #INSERT MODEL TRAINING HERE
        #model1 = f(train)
        #predict = predict(model1,test)

We can now save the data:

In [94]:
data.to_csv("grid_aug_train.csv")